In [8]:
import re
from dataclasses import dataclass
from pathlib import Path
from typing import List, Optional, Tuple, Dict, Any

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option("display.max_colwidth", 200)

def norm_col(name: str) -> str:
    """Maak kolomnamen klein en 'snake_case'."""
    name = str(name).strip().lower()
    name = re.sub(r"\s+", "_", name)
    name = name.replace("-", "_")
    return name


# data inladen & verkennen

In [9]:
# Pad naar je eigen dataset (pas aan indien nodig)
DATA_PATH = Path('Opgeschoonde_VKM_dataset.csv')

df_raw = pd.read_csv(DATA_PATH, sep=None, engine="python")  # autodetect ; of ,
df = df_raw.copy()

# Kolomnamen normaliseren
df.columns = [norm_col(c) for c in df.columns]

print("Vorm van de data:", df.shape)
display(df.head(2))
print("\nKolominformatie:")
print(df.info())
print("\nAantal missende waarden per kolom:")
print(df.isna().sum())


Vorm van de data: (211, 15)


,id,name,shortdescription,description,studycredit,location,contact_id,level,learningoutcomes,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,kennismaking met psychologie,"brein, gedragsbeinvloeding, ontwikkelingspsychologie, gespreksvoering en ontwikkelingsfasen.",in deze module leer je hoe je gedrag van jezelf en van anderen kunt begrijpen en beinvloeden. je maakt kennis met de basistheorie van psychologie. aan bod komen onderwerpen die te maken hebben met...,15,den bosch,58,nlqf5,a. je beantwoordt vragen in een meerkeuze kennistoets waarin je laat zien dat je de basis van de psychologie kunt reproduceren en begrijpt. je laat zien dat je gedrag van individuen en groepen in ...,"['brein', 'gedragsbeinvloeding', 'ontwikkelingspsychologie', 'gespreksvoering', 'en', 'ontwikkelingsfasen']",0.54,319,1,79,2025-12-24
1,160,learning and working abroad,"internationaal, persoonlijke ontwikkeling, verpleegkunde","studenten kiezen binnen de (stam) van de opleiding van verpleegkunde steeds vaker voor een stage in het buitenland, waarbij zij de beroepsprestaties graag in een internationale stagecontext willen...",15,den bosch,58,nlqf5,de student toont professioneel gedrag conform de beroepscode bij laagcomplexe zorgvragers en collega's in de zorgsetting.,"['internationaal', 'persoonlijke', 'ontwikkeling', 'verpleegkunde']",0.92,172,5,56,2025-12-20



Kolominformatie:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     211 non-null    int64  
 1   name                   211 non-null    object 
 2   shortdescription       211 non-null    object 
 3   description            211 non-null    object 
 4   studycredit            211 non-null    int64  
 5   location               211 non-null    object 
 6   contact_id             211 non-null    int64  
 7   level                  211 non-null    object 
 8   learningoutcomes       211 non-null    object 
 9   module_tags            211 non-null    object 
 10  interests_match_score  211 non-null    float64
 11  popularity_score       211 non-null    int64  
 12  estimated_difficulty   211 non-null    int64  
 13  available_spots        211 non-null    int64  
 14  start_date             211 non-null    o

# inhoudsprofiel van VKM

In [11]:
# Use explicit known columns (no ALIASES needed here)
text_columns = [c for c in ["short_description", "description", "module_tags", "learning_outcomes"] if c in df.columns]

if not text_columns:
    raise ValueError("Geen geschikte tekstkolommen gevonden om 'VKM_text' te maken. Controleer je data.")

# Combine available text columns into a single text field
df["VKM_text"] = df[text_columns].fillna("").astype(str).agg(" ".join, axis=1).str.replace(r"\s+", " ", regex=True).str.strip()

# Choose title/name column if present
title_col = "title" if "title" in df.columns else ("name" if "name" in df.columns else df.columns[0])

# show result
df[[title_col, "VKM_text"]].head()
# ...existing code...

,name,VKM_text
0,kennismaking met psychologie,in deze module leer je hoe je gedrag van jezelf en van anderen kunt begrijpen en beinvloeden. je maakt kennis met de basistheorie van psychologie. aan bod komen onderwerpen die te maken hebben met...
1,learning and working abroad,"studenten kiezen binnen de (stam) van de opleiding van verpleegkunde steeds vaker voor een stage in het buitenland, waarbij zij de beroepsprestaties graag in een internationale stagecontext willen..."
2,proactieve zorgplanning,"het jeroen bosch ziekenhuis wil graag samen met de opleiding verpleegkunde een module ontwikkelen, waarin de studenten de mogelijkheid krijgen om zich te verdiepen in de ziekenhuissetting. jbz sta..."
3,rouw en verlies,"in deze module wordt stil gestaan bij rouw en verlies, vanuit diverse invalshoeken waaronder de palliatieve zorg. thema's zoals oncologie kunnen hier een plaats krijgen (werkveld verpleegkunde vra..."
4,acuut complexe zorg,"in deze module kunnen studenten zich verdiepen in de acuut, complexe zorg binnen het verpleegkundig vakgebied. ['acute', 'zorg', 'complexiteit', 'ziekenhuis', 'revalidatie']"


In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

dutch_stopwords = stopwords.words('dutch')

tfidf = TfidfVectorizer(
    max_features=8000,
    ngram_range=(1, 2),
    stop_words=dutch_stopwords  # pass list of Dutch stopwords
)
X = tfidf.fit_transform(df["name"].fillna(""))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yazan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# kandidaat profiel

In [14]:
@dataclass
class CandidateProfile:
    interests_text: str
    preferred_location: Optional[str] = None # breda, Denbosch, Tilburg, breda en denbosch
    min_studycredits: Optional[float] = None # 15, 30
    max_difficulty: Optional[float] = None # 1, 2, 3, 4, 5
    moduletags_include: Optional[List[str]] = None  # bijv. ['data', 'analytics']
    min_level: Optional[str] = None #NLQF5, NLQF6


def profile_vector(profile: CandidateProfile):
    return tfidf.transform([profile.interests_text])


# Constraint-score

Naast inhoudelijke overeenkomst voegen we een simpele **constraint-score** toe op basis van:

- locatie (match / geen match),
- minimum studypunten,
- maximale moeilijkheid,
- module-tags.
- minimale level

In [16]:
def extract_numeric_series(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.replace(",", ".", regex=False)
    nums = pd.to_numeric(
        s.str.extract(r"([0-9]+\.?[0-9]*)")[0],
        errors="coerce"
    )
    return nums


def constraint_score(row: pd.Series, profile: CandidateProfile) -> Tuple[float, Dict[str, str]]:
 

    scores = []
    reasons: Dict[str, str] = {}

    # Locatie (check for a 'location' column)
    loc_col = "location" if "location" in row.index else None
    if profile.preferred_location and loc_col:
        job_loc = str(row[loc_col]).lower()
        pref = profile.preferred_location.lower()
        match = pref in job_loc
        scores.append(1.0 if match else 0.0)
        reasons["location"] = f"Locatie {'matcht' if match else 'matcht niet'}: kandidaat={profile.preferred_location}, vacature={row[loc_col]}"
    else:
        reasons["location"] = "Geen locatie-voorkeur opgegeven of kolom ontbreekt."

    # Minimum salaris (or fallback name check) — supports min_salary attribute if present
    sal_col = "salary" if "salary" in row.index else None
    min_salary = getattr(profile, "min_salary", None)
    if min_salary is not None and sal_col:
        salary_val = extract_numeric_series(pd.Series([row[sal_col]])).iloc[0]
        if pd.isna(salary_val):
            scores.append(0.5)  # onbekend salaris → neutraal
            reasons["studycredit"] = "studiepunten onbekend; neutrale score."
        else:
            match = salary_val >= min_salary
            scores.append(1.0 if match else 0.0)
            reasons["studycredit"] = f"Indicatie studiepunten {'voldoende' if match else 'lager dan voorkeur'} (≈ {salary_val})."
    else:
        reasons["studycredit"] = "Geen studiepunten-constraint of kolom."

    # Moeilijkheid
    diff_col = "estimated_difficulty" if "estimated_difficulty" in row.index else None
    if getattr(profile, "max_difficulty", None) is not None and diff_col:
        diff_val = extract_numeric_series(pd.Series([row[diff_col]])).iloc[0]
        if pd.isna(diff_val):
            scores.append(0.5)
            reasons["estimated_difficulty"] = "Moeilijkheid onbekend; neutrale score."
        else:
            match = diff_val <= getattr(profile, "max_difficulty")
            scores.append(1.0 if match else 0.0)
            reasons["estimated_difficulty"] = f"Moeilijkheid {'binnen' if match else 'boven'} voorkeursniveau (≈ {diff_val})."
    else:
        reasons["estimated_difficulty"] = "Geen moeilijkheids-constraint of kolom."

    # module tags — support module_include OR moduletags_include on profile
    role_col = "module_tags" if "module_tags" in row.index else None
    wanted = getattr(profile, "module_include", None) or getattr(profile, "moduletags_include", None)
    if wanted and role_col:
        tags = str(row[role_col]).lower()
        wanted = [w.lower() for w in wanted]
        match_count = sum(1 for w in wanted if w in tags)
        sc = min(1.0, match_count / max(1, len(wanted)))
        scores.append(sc)
        reasons["module_tags"] = f"Domeinmatch: {match_count} van {len(wanted)} gewenste termen gevonden."
    else:
        reasons["module_tags"] = "Geen domein- / tag-constraint opgegeven of kolom ontbreekt."

    if not scores:
        return 0.0, reasons

    return float(np.mean(scores)), reasons

## 7. Aanbevelingsfunctie

We combineren drie componenten in één eindscore:

- **content_sim**: cosine similarity tussen profieltekst en `job_text`;
- **constraint_score**: gemiddeld van de bovenstaande constraints;
- **popularity**: optioneel, als de dataset een populariteitsmaat bevat.

De gewichten `alpha`, `beta`, `gamma` moeten optellen tot 1.

In [19]:
def first_existing_col(df: pd.DataFrame, candidates: list) -> Optional[str]:
    for c in candidates:
        if c in df.columns:
            return c
    return None

def recommend(
    profile: CandidateProfile,
    k: int = 10,
    alpha: float = 0.7,   # gewicht content
    beta: float = 0.2,    # gewicht constraints
    gamma: float = 0.1    # gewicht populariteit
) -> pd.DataFrame:
    if not np.isclose(alpha + beta + gamma, 1.0):
        raise ValueError("alpha + beta + gamma moeten samen 1.0 zijn.")

    p_vec = profile_vector(profile)
    content_scores = cosine_similarity(p_vec, X).flatten()

    pop_col = first_existing_col(df, ["popularity_score"])
    if pop_col:
        pop_raw = extract_numeric_series(df[pop_col]).fillna(0.0)
        pop_scaled = (pop_raw - pop_raw.min()) / (pop_raw.max() - pop_raw.min() + 1e-9)
    else:
        pop_scaled = pd.Series(0.0, index=df.index)

    rows = []
    title_col = first_existing_col(df, ["name"]) or df.columns[0]
    loc_col = first_existing_col(df, ["location"])

    for idx, row in df.iterrows():
        cstr_score, reasons = constraint_score(row, profile)
        c_score = float(content_scores[idx])
        pop_score = float(pop_scaled.loc[idx])

        final_score = alpha * c_score + beta * cstr_score + gamma * pop_score

        rows.append({
            "index": idx,
            "name": row.get(title_col, ""),
            "location": row.get(loc_col, ""),
            "final_score": final_score,
            "content_sim": c_score,
            "constraint_score": cstr_score,
            "popularity_score": pop_score,
            "constraint_reasons": reasons,
        })

    rec_df = pd.DataFrame(rows).sort_values("final_score", ascending=False).head(k).reset_index(drop=True)
    return rec_df

## 8. Diversiteit van aanbevelingen & uitleg

We voegen twee eenvoudige extra functies toe:

- `diversity_score(indices)`: 1 – gemiddelde onderlinge cosine similarity.
- `explain_overlap(...)`: overlappende trefwoorden tussen profiel en vacaturetekst.

In [20]:
def diversity_score(indices: List[int]) -> float:
    if len(indices) < 2:
        return 0.0
    sub = X[indices]
    sim = cosine_similarity(sub)
    n = sim.shape[0]
    mask = np.triu(np.ones_like(sim, dtype=bool), k=1)
    if mask.sum() == 0:
        return 0.0
    avg_sim = sim[mask].mean()
    return float(1.0 - avg_sim)

def explain_overlap(profile_text: str, job_text: str, top_k: int = 8) -> List[str]:
    def tokenize(s: str) -> List[str]:
        return re.findall(r"\w+", s.lower())

    p_tokens = tokenize(profile_text)
    j_tokens = tokenize(job_text)

    common = [t for t in j_tokens if t in set(p_tokens)]
    counts: Dict[str, int] = {}
    for t in common:
        counts[t] = counts.get(t, 0) + 1

    sorted_terms = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    return [w for w, _ in sorted_terms[:top_k]]


## 9. Demo: aanbevelingen voor een voorbeeldprofiel

In [27]:
demo_profile = CandidateProfile(
    interests_text="data analytics machine learning python",
    preferred_location="breda",
    min_studycredits=15,
    max_difficulty=4.0,
    moduletags_include=["data", "analytics"]
)

demo_recs = recommend(demo_profile, k=5, alpha=0.7, beta=0.2, gamma=0.1)
demo_recs


,index,name,location,final_score,content_sim,constraint_score,popularity_score,constraint_reasons
0,171,molecular modeling & data-driven analysis,breda,0.394979,0.186511,0.833333,0.977551,"{'location': 'Locatie matcht: kandidaat=breda, vacature=breda', 'studycredit': 'Geen studiepunten-constraint of kolom.', 'estimated_difficulty': 'Moeilijkheid binnen voorkeursniveau (≈ 1).', 'modu..."
1,166,data/visualisatie/automatisering (tech & data) (ct),breda en den bosch,0.388130,0.315211,0.833333,0.008163,"{'location': 'Locatie matcht: kandidaat=breda, vacature=breda en den bosch', 'studycredit': 'Geen studiepunten-constraint of kolom.', 'estimated_difficulty': 'Moeilijkheid binnen voorkeursniveau (..."
2,73,module 2.3 data gestuurd ontwerp (data driven design),den bosch en tilburg,0.339706,0.308909,0.500000,0.234694,"{'location': 'Locatie matcht niet: kandidaat=breda, vacature=den bosch en tilburg', 'studycredit': 'Geen studiepunten-constraint of kolom.', 'estimated_difficulty': 'Moeilijkheid binnen voorkeursn..."
3,127,meaningful data design,den bosch,0.280980,0.260487,0.166667,0.653061,"{'location': 'Locatie matcht niet: kandidaat=breda, vacature=den bosch', 'studycredit': 'Geen studiepunten-constraint of kolom.', 'estimated_difficulty': 'Moeilijkheid boven voorkeursniveau (≈ 5)...."
4,158,chemie & gezondheid,breda en den bosch,0.266667,0.000000,0.833333,1.000000,"{'location': 'Locatie matcht: kandidaat=breda, vacature=breda en den bosch', 'studycredit': 'Geen studiepunten-constraint of kolom.', 'estimated_difficulty': 'Moeilijkheid binnen voorkeursniveau (..."


In [26]:
# Diversiteitsscore van de aanbevelingen
indices = demo_recs["index"].tolist()
print("Diversiteitsscore (0 = zeer gelijkend, 1 = zeer divers):", diversity_score(indices))

# Toon overlappende termen voor de eerste 3 aanbevelingen
title_col = "name"
for _, row in demo_recs.head(3).iterrows():
    idx = row["index"]
    job_text = df.loc[idx, "VKM_text"]
    overlap = explain_overlap(demo_profile.interests_text, job_text, top_k=8)
    print(f"\nVKM: {row[title_col]}")
    print("Overlappende trefwoorden:", ", ".join(overlap) if overlap else "(geen expliciete overlap gevonden)")


Diversiteitsscore (0 = zeer gelijkend, 1 = zeer divers): 0.8728944546525194

VKM: molecular modeling & data-driven analysis
Overlappende trefwoorden: data, python, machine, learning

VKM: data/visualisatie/automatisering (tech & data) (ct)
Overlappende trefwoorden: data

VKM: module 2.3 data gestuurd ontwerp (data driven design)
Overlappende trefwoorden: data
